<a href="https://www.kaggle.com/code/shravankumar147/real-time-ai-app-with-groq-api-and-langchain?scriptVersionId=209759253" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Setting Up

In [2]:
%%capture
   
%pip install langchain langchain-community 
%pip install langchainhub 
%pip install langchain-chroma 
%pip install langchain-groq
%pip install langchain-huggingface
%pip install gradio

In [4]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
groq_api_key = user_secrets.get_secret("Groq")

## Groq Python API

In [5]:
from groq import Groq

client = Groq(    
   api_key=groq_api_key,
)


chat_streaming = client.chat.completions.create( 
    messages=[        
       {"role": "system", "content": "You are a professional Data Engineer."},        
       {"role": "user", "content": "Can you explain how the data lake works?"},    
    ],    
    model="llama-3.1-8b-instant", 
    temperature=0.3, 
    max_tokens=1200, 
    top_p=1, 
    stop=None, 
    stream=True,
)

for chunk in chat_streaming: 
    print(chunk.choices[0].delta.content, end="")

A data lake is a centralized repository that stores raw, unprocessed data in its native format, allowing for easy storage and querying of large amounts of data. Here's how a data lake works:

**Key Characteristics:**

1. **Schema-on-Read**: Unlike traditional data warehouses, which require a predefined schema, a data lake stores data in its native format, and the schema is defined at query time.
2. **Unprocessed Data**: Data is stored in its raw, unprocessed form, without any transformations or aggregations.
3. **Centralized Repository**: Data is stored in a single, centralized location, making it easier to manage and access.
4. **Scalability**: Data lakes are designed to handle large amounts of data and scale horizontally to accommodate growing data volumes.

**Data Lake Architecture:**

A typical data lake architecture consists of the following components:

1. **Data Ingestion Layer**: This layer is responsible for collecting data from various sources, such as databases, APIs, and fi

## Initiating LLM

In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant",api_key=groq_api_key)

## Initiating Embedding Model

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
embed_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

## Loading the CSV files

In [8]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("/kaggle/input/star-wars/csv", glob="**/*.csv", loader_cls=CSVLoader)

data = loader.load()

In [9]:
len(data)

488

## Setting up VectorStore

In [10]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=data,
    embedding=embed_model,
    persist_directory="/kaggle/working/KaggleX_Starwars",
)


In [11]:
query = "Which battle resulted in Rebel Victory?"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

id: 3
name: Battle of Endor
location: Endor
date: 4 ABY
result: Rebel Victory
participants: Rebel Alliance, Galactic Empire


In [12]:
len(docs)

4

## Building Components for RAG Chain

In [13]:
retriever = vectorstore.as_retriever()

In [14]:
from langchain_core.prompts import PromptTemplate

template = ("""You are a Star Wars assistant for answering questions. 
    Use the provided context to answer the question. 
    If you don't know the answer, say so. Explain your answer in detail. 
    Do not discuss the context in your response; just provide the answer directly.

    Context: {context}

    Question: {question}
    
    Answer:""")

rag_prompt = PromptTemplate.from_template(template)

In [15]:
rag_prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are a Star Wars assistant for answering questions. \n    Use the provided context to answer the question. \n    If you don't know the answer, say so. Explain your answer in detail. \n    Do not discuss the context in your response; just provide the answer directly.\n\n    Context: {context}\n\n    Question: {question}\n    \n    Answer:")

## Building the RAG Chain

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

## Testing the Chain

In [17]:
from IPython.display import display, Markdown

response = rag_chain.invoke("Which battle resulted in Rebel Victory?")
Markdown(response)

The battles that resulted in Rebel Victory are the Battle of Endor and the Battle of Yavin.

In [18]:
query = "What is the timeline of the Starwars?"

for chunk in rag_chain.stream(query):
            print(chunk, end="")

The timeline of the Star Wars is complex and spans thousands of years. However, based on the provided information, we can establish a rough timeline of events.

Here's a detailed breakdown:

- The events of Episode I: The Phantom Menace take place approximately 32 years before the events of Episode IV: A New Hope.
- The events of Episode II, III, and IV occur in a relatively short period, roughly spanning a decade.
- The events of Episode V: The Empire Strikes Back occur approximately 3 years after the events of Episode IV: A New Hope.
- The events of Episode VI: Return of the Jedi occur approximately 3 years after the events of Episode V: The Empire Strikes Back (although this is not explicitly stated in the provided information, it is widely known).
- The Battle of Endor, as mentioned in the provided information, occurs in 4 ABY (After the Battle of Yavin), which is after the events of Episode VI: Return of the Jedi.

A more detailed timeline would include events from the prequel and

In [19]:
import gradio as gr


def rag_memory(text):
    response = rag_chain.invoke(text)
    return response


description = "Real-time AI App with Groq API and LangChain"
demo = gr.Interface(
    description=description,
    fn=rag_memory,
    inputs="text",
    outputs="text",
    live=True,
    batch=True,
    max_batch_size=10000,
)
demo.queue(max_size=300000)
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://16689f9e1e652dc389.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
